In [17]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
# membaca data
header_names=['ItemID','UserID','Rating']
df = pd.read_csv("Books Sorted.csv",delimiter=',',header=None,names=header_names)

In [19]:
# menyeleksi user yang rajin memberikan rating
combine_books_rating_user=df.dropna(axis=0,subset=['UserID'])
books_ratingcount_user=(combine_books_rating_user.
                  groupby(by=['UserID'])['Rating'].
                  count().
                  reset_index().
                  rename(columns={'Rating':'totalRatingCountuser'})
                  [['UserID','totalRatingCountuser']])
rating_with_totalRatingCount_user=combine_books_rating_user.merge(books_ratingcount_user,left_on='UserID',right_on='UserID',how='left')
#Ceritanya kita cuma meninjau user-user yang rajin rating dengan minimal memberikan rating sebesar rate_treshold_user
rate_treshold_user=10
rating_popular_book_user=rating_with_totalRatingCount_user.query('totalRatingCountuser>=@rate_treshold_user')
rating_popular_books_user=rating_popular_book_user.drop(['totalRatingCountuser'],axis=1)
rating_popular_books_user.shape

(48472, 3)

In [20]:
# menyeleksi item yang populer
combine_books_rating_item=rating_popular_books_user.dropna(axis=0,subset=['ItemID'])
books_ratingcount_item=(combine_books_rating_item.
                  groupby(by=['ItemID'])['Rating'].
                  count().
                  reset_index().
                  rename(columns={'Rating':'totalRatingCountitem'})
                  [['ItemID','totalRatingCountitem']])
rating_with_totalRatingCount_item=combine_books_rating_item.merge(books_ratingcount_item,left_on='ItemID',right_on='ItemID',how='left')
#Ceritanya kita cuma meninjau buku-buku yang popular dengan minimal diberikan rating sebesar popularity_treshold_item
popularity_treshold_item=20
rating_popular_book_item=rating_with_totalRatingCount_item.query('totalRatingCountitem>=@popularity_treshold_item')
rating_popular_books=rating_popular_book_item.drop(['totalRatingCountitem'],axis=1)
rating_popular_books.shape

(32333, 3)

In [21]:
# Menghitung rating yang diberikan user dengan rataan rating yang diberikan oleh user adalah 0
Mean_user=rating_popular_books.groupby(by="UserID",as_index=False)['Rating'].mean()
Rating_avg=pd.merge(rating_popular_books,Mean_user,on='UserID')
Rating_avg['adg_rating']=Rating_avg['Rating_x']-Rating_avg['Rating_y']

In [22]:
#matrix User Item rata-rata di nol & matriks user item rating
matrix_df=Rating_avg.pivot_table(index='UserID',columns='ItemID',values='Rating_x').fillna(0)
books_matrix_df=Rating_avg.pivot_table(index='UserID',columns='ItemID',values='adg_rating').fillna(0)

In [23]:
# menghitung cosinus/similaritas antar user dalam matriks user-user
cosine=cosine_similarity(books_matrix_df)
np.fill_diagonal(cosine,-2)
similarity_with_user=pd.DataFrame(cosine,index=books_matrix_df.index)
similarity_with_user.columns=books_matrix_df.index
similarity_with_user.head()

UserID,A102Z3T7NSM5KC,A103M7E0BSFC83,A105E427BB6J65,A10A1S5NAQBT21,A10BZSGALQPS0V,A10F2RGD2D36ZI,A10K3DLOEVMKW3,A10LKYQ1D8LPK8,A10LWBOIZCF2QT,A10Q15TOZBPR5N,...,AZ85B5Q1UEH5U,AZDCCMYLOEU3X,AZFX7FLEHCWXM,AZJHIRNFI04WK,AZK12FBX7X7LN,AZM9G2E1UMKRZ,AZP6KZVJIPROJ,AZPMV1PM3ISSL,AZQS0Y3RKFVIJ,AZUNT3QP2CWTL
UserID,,,,,,,,,,,,,,,,,,,,,
A102Z3T7NSM5KC,-2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.473326,0.0,0.0
A103M7E0BSFC83,0.0,-2.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
A105E427BB6J65,0.0,0.0,-2.0,0.0,0.0,0.0,0.194701,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
A10A1S5NAQBT21,0.0,0.0,0.0,-2.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
A10BZSGALQPS0V,0.0,0.0,0.0,0.0,-2.0,0.0,0.000000,0.0,0.154499,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [24]:
# konvert dataframe yang akan digunakan untuk prediksi rating menjadi dictionary
# Matriks rating
R=matrix_df.to_dict(orient='index')
RT=matrix_df.to_dict() #transposenya
# Matriks rating dengan rataan di 0
RM=books_matrix_df.to_dict(orient='index')
#Matrix similaritas antar user
U=similarity_with_user.to_dict(orient='index')
# Rataan rating masing-masing user
Ap=Mean_user.set_index('UserID').T.to_dict()
A={}
for i in Ap.keys():
    for j in Ap[i].keys():
        A[i]=Ap[i][j]

In [27]:
# Input nilai K maksimal adalah sebesar popularity_treshold_item-1
K= int(input('masukkan nilai K: '))
#memprediksi rating
RP = {} #matriks rating prediksi
I = {} #Siapa saja yang beli item ke-i
Rm={} #prediksi rating yang sudah di rating
for j in RT.keys():
    I[j]=[]
    for i in RT[j].keys():
        if R[i][j]!=0:
            I[j].append(i) #mengetahui user mana saja yang memberikan rating terhadap item-j
for i in R.keys():
    RP[i]={}
    for j in R[i].keys():
        if R[i][j]==0:
            x = []
            for k in I[j]:
                x.append([k,U[i][k]]) #Menambahkan kode user serta similaritas user yang beli item ke-j
            y=sorted(x,key=lambda x:x[1],reverse=True) # Mengurutkan sub-array yang ada di x berdasarkan nilai similaritas
            s = 0
            count=0
            for k in range(min(K,len(x))):
                m = y[k][0] #Input kode user
                s = s + U[i][m]*RM[m][j] #Menghitung bagian pembilang pada rumus rating prediksi
                count=count+abs(U[i][m]) #Menghitung bagian penyebut pada rumus rating prediksi
            if count==0:
                RP[i][j] = A[i] #jaga-jaga jika similaritas pengguna lain semuanya nol agar tidak terjadi pembagian dengan nol
            else:
                RP[i][j] = A[i]+s/count

masukkan nilai K: 3


In [30]:
# Memberikan rekomendasi Item untuk suatu UserID tertentu
UserID= str(input('masukkan UserID: '))
D=RP[UserID]
L=sorted(D.items(), key=lambda kv:kv[1],reverse=True)
print('Rekomendasi ItemID untuk UserID ',UserID,'adalah :')
for k in range(5):
    print('Top',k+1,'-->',L[k][0])

masukkan UserID: A0192904YO1UGFMZVKRI


KeyError: 'A0192904YO1UGFMZVKRI'

In [11]:
# Menghitung nilai RMSE
for i in R.keys():
    Rm[i]={}
    for j in R[i].keys():
        if R[i][j]!=0:
            x = []
            for k in I[j]:
                x.append([k,U[i][k]]) #Similaritas user2 yang beli item ke-j
            y=sorted(x,key=lambda x:x[1],reverse=True)
            s = 0
            count=0
            for k in range(min(K,len(x))):
                m = y[k][0]
                s = s + U[i][m]*RM[m][j]
                count=count+abs(U[i][m])
            if count==0:
                Rm[i][j] = A[i]
            else:
                Rm[i][j] = A[i]+s/count
N=0
s=0
for i in Rm.keys():
    for j in Rm[i].keys():
        s=s+(R[i][j]-Rm[i][j])**2
        N=N+1
RMSE=(s/N)**(0.5)
RMSE

0.578579454051359

28342